In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json
import pandas as pd

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

# Load data

In [2]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_bin.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)


In [3]:
row_names = list(edges.keys())
n_state = 2 # binary states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [4]:
#origin = 'n1'
origin = 'n52'
dests = ['n22', 'n66']
sys_surv_st = 1

def s_fun(comps_st):
    travel_time, sys_st, info = fbg.eval_travel_time_to_nearest(comps_st, G_base, origin, dests,
                                                         avg_speed=60, # km/h
                                                         target_max = [1.5, 0.5], # hours: it shouldn't take longer than this compared to the original travel time
                                                         length_attr = 'length_km')
    if sys_st >= sys_surv_st:
       path = info['path_filtered_edges'] 
       min_comps_st = {eid: ('>=', 1) for eid in path} # edges in the path are working
       min_comps_st['sys'] = ('>=', sys_st) # system edge is also working
    else:
        min_comps_st = None
    return travel_time, sys_st, min_comps_st

In [5]:
TSUMPATH = Path("tsum_res") 

rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

C:\Users\jb622s\AppData\Local\Temp\ipykernel_1680\2953922633.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv

# Calculate system probabilities

## Marginal probability

In [6]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun,
    sys_surv_st = sys_surv_st
)
print(f"P(sys >= {sys_surv_st}) = {pr_cond['survival']:.3e}")
print(f"P(sys <= {sys_surv_st-1} ) = {pr_cond['failure']:.3e}\n")

P(sys >= 1) = 9.988e-01
P(sys <= 0 ) = 1.229e-03



## Conditional probability given "one" component's survival

In [7]:
for x in row_names:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun,
        sys_surv_st = sys_surv_st
    )
    print(f"P(sys >= {sys_surv_st} | {x}=1) = {pr_cond['survival']:.3e}")
    print(f"P(sys <= {sys_surv_st-1} | {x}=1) = {pr_cond['failure']:.3e}\n")

Eval P(sys | e0001=1)
P(sys >= 1 | e0001=1) = 9.988e-01
P(sys <= 0 | e0001=1) = 1.242e-03

Eval P(sys | e0002=1)
P(sys >= 1 | e0002=1) = 9.988e-01
P(sys <= 0 | e0002=1) = 1.188e-03

Eval P(sys | e0003=1)
P(sys >= 1 | e0003=1) = 9.988e-01
P(sys <= 0 | e0003=1) = 1.216e-03

Eval P(sys | e0004=1)
P(sys >= 1 | e0004=1) = 9.988e-01
P(sys <= 0 | e0004=1) = 1.248e-03

Eval P(sys | e0005=1)
P(sys >= 1 | e0005=1) = 9.988e-01
P(sys <= 0 | e0005=1) = 1.212e-03

Eval P(sys | e0006=1)
P(sys >= 1 | e0006=1) = 9.988e-01
P(sys <= 0 | e0006=1) = 1.198e-03

Eval P(sys | e0007=1)
P(sys >= 1 | e0007=1) = 9.988e-01
P(sys <= 0 | e0007=1) = 1.207e-03

Eval P(sys | e0008=1)
P(sys >= 1 | e0008=1) = 9.988e-01
P(sys <= 0 | e0008=1) = 1.241e-03

Eval P(sys | e0009=1)
P(sys >= 1 | e0009=1) = 9.987e-01
P(sys <= 0 | e0009=1) = 1.253e-03

Eval P(sys | e0010=1)
P(sys >= 1 | e0010=1) = 9.988e-01
P(sys <= 0 | e0010=1) = 1.232e-03

Eval P(sys | e0011=1)
P(sys >= 1 | e0011=1) = 9.987e-01
P(sys <= 0 | e0011=1) = 1.255e-03


KeyboardInterrupt: 

# Get multi-state probabilities

## First load rules for all states

In [8]:
TSUMPATH = Path("tsum_res") 
rules_dict_mat_surv = {}
rules_dict_mat_fail = {}

for sys_surv_st in [1, 2]:  # either 1 or 2
    rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
    rules_mat_surv = rules_mat_surv.to(device)
    rules_dict_mat_surv[sys_surv_st] = rules_mat_surv
    rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
    rules_mat_fail = rules_mat_fail.to(device)
    rules_dict_mat_fail[sys_surv_st] = rules_mat_fail

C:\Users\jb622s\AppData\Local\Temp\ipykernel_1680\855290552.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_

## Marginal probability

In [10]:

# Initialize empty list to store the results
results = []

# Calculate probabilities
cond_probs = tsum.get_comp_cond_sys_prob_multi(
                rules_dict_mat_surv,
                rules_dict_mat_fail,
                probs,
                comps_st_cond = {},
                row_names = row_names,
                s_fun = s_fun
            )

# Print results
print("P(sys):", cond_probs)

# Append data as a dictionary to the list
results.append({"System failure": cond_probs[0],
                "Partial failure": cond_probs[1],
                "Survival": cond_probs[2]
                })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

# Save to a JSON file
df_results.to_json("post-processing/sys_probs.json", orient="records", indent=4)

print("\nData saved to post-processing/sys_probs.json")

P(sys): {0: 0.001188, 1: 0.000131, 2: 0.998681}


## Conditional probability given "one" component's survival
State 0: System failure\
State 1: Partial failure\
State 2: System survival

In [11]:
# Initialize empty list to store the results
results = []

for x in row_names:
    # Calculate probabilities
    cond_probs = tsum.get_comp_cond_sys_prob_multi(
                    rules_dict_mat_surv,
                    rules_dict_mat_fail,
                    probs,
                    comps_st_cond = {x: 0}, # 1: survival, 0: failure
                    row_names = row_names,
                    s_fun = s_fun
                )

    # Print results
    print(f"P(sys | {x}=0):", cond_probs)

    # Append data as a dictionary to the list
    results.append({"Component": x,
                    "System failure": cond_probs[0],
                    "Partial failure": cond_probs[1],
                    "Survival": cond_probs[2]
                    })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

# Save to a JSON file
df_results.to_json("post-processing/cond_sys_probs.json", orient="records", indent=4)

print("\nData saved to post-processing/cond_sys_probs.json")

P(sys | e0001=1): {0: 0.001241, 1: 0.000138, 2: 0.998621}
P(sys | e0002=1): {0: 0.001242, 1: 0.000124, 2: 0.998634}
P(sys | e0003=1): {0: 0.001224, 1: 0.000134, 2: 0.998642}
P(sys | e0004=1): {0: 0.001266, 1: 0.000144, 2: 0.99859}
P(sys | e0005=1): {0: 0.001217, 1: 0.000135, 2: 0.998648}
P(sys | e0006=1): {0: 0.001233, 1: 0.00016, 2: 0.998607}
P(sys | e0007=1): {0: 0.001226, 1: 0.000139, 2: 0.998635}
P(sys | e0008=1): {0: 0.001201, 1: 0.000141, 2: 0.998658}
P(sys | e0009=1): {0: 0.00128, 1: 0.000154, 2: 0.998566}
P(sys | e0010=1): {0: 0.001206, 1: 0.000155, 2: 0.998639}
P(sys | e0011=1): {0: 0.001245, 1: 0.000149, 2: 0.998606}
P(sys | e0012=1): {0: 0.001245, 1: 0.000138, 2: 0.998617}
P(sys | e0013=1): {0: 0.001185, 1: 0.000136, 2: 0.998679}
P(sys | e0014=1): {0: 0.001251, 1: 0.00014, 2: 0.998609}
P(sys | e0015=1): {0: 0.001236, 1: 0.000156, 2: 0.998608}
P(sys | e0016=1): {0: 0.001217, 1: 0.000125, 2: 0.998658}
P(sys | e0017=1): {0: 0.001191, 1: 0.000141, 2: 0.998668}
P(sys | e0018=1): 